In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import os
import cv2

In [14]:
directory = r"C:\Users\sivav\Edges"
categories = ['Horizontal', 'Vertical']
data = []
labels = []

for category in categories:
    path = os.path.join(directory, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        data.append(image)
        if(category == 'Horizontal'):
            labels.append(0)
        else:
            labels.append(1)
            
data = np.asarray(data)
labels = np.asarray(labels)
labels = labels.reshape((10000,1))

In [16]:
batch_size = 128
tensor_x = torch.Tensor(data) 
tensor_y = torch.Tensor(labels)
my_dataset = TensorDataset(tensor_x,tensor_y)
train_loader = DataLoader(my_dataset ,batch_size = batch_size, shuffle = True)

(10000, 1)